# 04 - Function Calling

**Master production-ready function calling** across different LLM providers.

## Learning Objectives

By the end of this notebook, you will:
- Understand OpenAI's function calling format
- Understand Anthropic's tool use format
- Handle multi-turn conversations with tools
- Implement proper error handling

## Table of Contents

1. [OpenAI Function Calling](#openai)
2. [Anthropic Tool Use](#anthropic)
3. [Multi-Turn Conversations](#multi-turn)
4. [Error Handling](#errors)
5. [Using Our LLM Client](#client)
6. [Exercises](#exercises)
7. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

# Import both clients
from openai import OpenAI

openai_client = OpenAI()

# Check for Anthropic
try:
    from anthropic import Anthropic
    anthropic_client = Anthropic()
    has_anthropic = True
except:
    has_anthropic = False

print("Setup complete!")
print(f"  OpenAI: Available")
print(f"  Anthropic: {'Available' if has_anthropic else 'Not configured'}")

In [ ]:
# GUIDED: Define some tools we'll use throughout

def get_weather(location: str, unit: str = "celsius") -> dict:
    """Mock weather function."""
    import random
    temps = {"celsius": random.randint(15, 30), "fahrenheit": random.randint(59, 86)}
    return {
        "location": location,
        "temperature": temps.get(unit, temps["celsius"]),
        "unit": unit,
        "condition": random.choice(["sunny", "cloudy", "rainy"])
    }

def search_web(query: str, num_results: int = 3) -> list:
    """Mock search function."""
    return [
        {"title": f"Result 1 for: {query}", "url": "https://example.com/1"},
        {"title": f"Result 2 for: {query}", "url": "https://example.com/2"},
        {"title": f"Result 3 for: {query}", "url": "https://example.com/3"}
    ][:num_results]

def calculate(expression: str) -> str:
    """Safe calculation."""
    allowed = set('0123456789+-*/.() ')
    if not all(c in allowed for c in expression):
        return "Error: Invalid expression"
    try:
        return str(eval(expression))
    except:
        return "Error: Could not evaluate"

# Map function names to functions
FUNCTIONS = {
    "get_weather": get_weather,
    "search_web": search_web,
    "calculate": calculate
}

print("Functions defined:", list(FUNCTIONS.keys()))

---
## 1. OpenAI Function Calling <a id='openai'></a>

OpenAI's format uses `tools` with `function` type definitions.

In [ ]:
# GUIDED: OpenAI tool definitions

openai_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g., 'Tokyo' or 'New York'"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform mathematical calculations",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Math expression like '2 + 2' or '10 * 5'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

print(f"Defined {len(openai_tools)} OpenAI tools")

In [ ]:
# GUIDED: Make a request with tools

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What's the weather in Paris?"}
    ],
    tools=openai_tools,
    tool_choice="auto"  # or "required" to force tool use
)

message = response.choices[0].message

print("Response:")
print(f"  Content: {message.content}")
print(f"  Tool Calls: {message.tool_calls}")
print(f"  Finish Reason: {response.choices[0].finish_reason}")

In [ ]:
# GUIDED: Handle the tool call

if message.tool_calls:
    for tool_call in message.tool_calls:
        print(f"\nTool Call:")
        print(f"  ID: {tool_call.id}")
        print(f"  Function: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
        
        # Parse arguments and call function
        args = json.loads(tool_call.function.arguments)
        func = FUNCTIONS[tool_call.function.name]
        result = func(**args)
        
        print(f"  Result: {result}")

---
## 2. Anthropic Tool Use <a id='anthropic'></a>

Anthropic uses a slightly different format with `input_schema` instead of `parameters`.

In [ ]:
# GUIDED: Anthropic tool definitions

anthropic_tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name, e.g., 'Tokyo' or 'New York'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate",
        "description": "Perform mathematical calculations",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Math expression like '2 + 2' or '10 * 5'"
                }
            },
            "required": ["expression"]
        }
    }
]

print(f"Defined {len(anthropic_tools)} Anthropic tools")

In [ ]:
# GUIDED: Make a request with Anthropic

if has_anthropic:
    response = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=anthropic_tools,
        messages=[
            {"role": "user", "content": "What's the weather in London?"}
        ]
    )
    
    print("Response:")
    print(f"  Stop Reason: {response.stop_reason}")
    
    for block in response.content:
        print(f"  Block Type: {block.type}")
        if block.type == "text":
            print(f"    Text: {block.text}")
        elif block.type == "tool_use":
            print(f"    Tool: {block.name}")
            print(f"    ID: {block.id}")
            print(f"    Input: {block.input}")
else:
    print("Anthropic not configured. Skipping this example.")

### Format Comparison

| Aspect | OpenAI | Anthropic |
|--------|--------|-----------|
| Tool wrapper | `{"type": "function", "function": {...}}` | Direct tool object |
| Parameters field | `parameters` | `input_schema` |
| Tool call ID | `tool_calls[].id` | `content[].id` (when type=tool_use) |
| Arguments | `tool_calls[].function.arguments` (string) | `content[].input` (dict) |
| Stop reason | `finish_reason: "tool_calls"` | `stop_reason: "tool_use"` |

---
## 3. Multi-Turn Conversations <a id='multi-turn'></a>

Real applications require sending tool results back to the LLM.

In [ ]:
# GUIDED: Complete multi-turn flow with OpenAI

def complete_with_tools(user_message: str, tools: list, max_iterations: int = 5):
    """
    Complete a conversation, handling any tool calls.
    """
    messages = [{"role": "user", "content": user_message}]
    
    for i in range(max_iterations):
        print(f"\n--- Iteration {i+1} ---")
        
        # Make API call
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        finish_reason = response.choices[0].finish_reason
        
        print(f"Finish Reason: {finish_reason}")
        
        # If no tool calls, we're done
        if finish_reason == "stop" or not message.tool_calls:
            print(f"Final Response: {message.content}")
            return message.content
        
        # Add assistant message with tool calls
        messages.append(message)
        
        # Process each tool call
        for tool_call in message.tool_calls:
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            
            print(f"Calling: {func_name}({args})")
            
            # Execute the function
            if func_name in FUNCTIONS:
                result = FUNCTIONS[func_name](**args)
            else:
                result = f"Error: Unknown function {func_name}"
            
            print(f"Result: {result}")
            
            # Add tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result) if isinstance(result, (dict, list)) else str(result)
            })
    
    return "Max iterations reached"

# Test it
result = complete_with_tools(
    "What's the weather in Tokyo? Also, what's 15 * 23?",
    openai_tools
)

### Key Points for Multi-Turn

1. **Add the assistant message** (including tool_calls) to history
2. **Add tool results** with matching `tool_call_id`
3. **Continue until** finish_reason is "stop"
4. **Set a max iterations** limit to prevent infinite loops

---
## 4. Error Handling <a id='errors'></a>

Robust tool handling requires proper error management.

In [ ]:
# GUIDED: Error handling patterns

def safe_execute_tool(func_name: str, args: dict) -> dict:
    """
    Safely execute a tool with error handling.
    """
    result = {
        "success": False,
        "result": None,
        "error": None
    }
    
    # Check if function exists
    if func_name not in FUNCTIONS:
        result["error"] = f"Unknown function: {func_name}"
        return result
    
    # Try to execute
    try:
        func = FUNCTIONS[func_name]
        output = func(**args)
        result["success"] = True
        result["result"] = output
    except TypeError as e:
        result["error"] = f"Invalid arguments: {str(e)}"
    except Exception as e:
        result["error"] = f"Execution error: {str(e)}"
    
    return result

# Test error handling
print("Valid call:")
print(safe_execute_tool("get_weather", {"location": "Paris"}))

print("\nUnknown function:")
print(safe_execute_tool("unknown_func", {}))

print("\nMissing argument:")
print(safe_execute_tool("get_weather", {}))  # Missing required 'location'

In [ ]:
# GUIDED: Validation before execution

def validate_arguments(func_name: str, args: dict, tool_defs: list) -> tuple[bool, str]:
    """
    Validate arguments against tool definition.
    """
    # Find the tool definition
    tool_def = None
    for tool in tool_defs:
        if tool["function"]["name"] == func_name:
            tool_def = tool["function"]
            break
    
    if not tool_def:
        return False, f"Unknown tool: {func_name}"
    
    params = tool_def.get("parameters", {})
    required = params.get("required", [])
    properties = params.get("properties", {})
    
    # Check required parameters
    for req in required:
        if req not in args:
            return False, f"Missing required parameter: {req}"
    
    # Check enum values
    for key, value in args.items():
        if key in properties:
            prop = properties[key]
            if "enum" in prop and value not in prop["enum"]:
                return False, f"Invalid value for {key}: {value}. Must be one of {prop['enum']}"
    
    return True, "Valid"

# Test validation
print("Valid arguments:")
print(validate_arguments("get_weather", {"location": "Paris", "unit": "celsius"}, openai_tools))

print("\nMissing required:")
print(validate_arguments("get_weather", {"unit": "celsius"}, openai_tools))

print("\nInvalid enum:")
print(validate_arguments("get_weather", {"location": "Paris", "unit": "kelvin"}, openai_tools))

---
## 5. Using Our LLM Client <a id='client'></a>

Our `LLMClient` abstracts away provider differences.

In [ ]:
# GUIDED: Use our unified client

from src.llm_client import LLMClient, Message
from src.tool_registry import ToolRegistry, Tool

# Create client
client = LLMClient(provider="openai", model="gpt-4o-mini")

# Create tools
registry = ToolRegistry()

registry.register(Tool(
    name="get_weather",
    description="Get current weather for a location",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City name"}
        },
        "required": ["location"]
    },
    function=get_weather
))

print("Client and tools ready!")

In [ ]:
# GUIDED: Use client with tools

# Make request with tools
response = client.chat(
    messages=[Message(role="user", content="What's the weather in Berlin?")],
    tools=registry.to_openai_format()
)

print("Response:")
print(f"  Content: {response.content}")
print(f"  Has tool calls: {response.has_tool_calls}")

if response.has_tool_calls:
    for tc in response.tool_calls:
        print(f"\nTool Call: {tc.name}")
        print(f"  Arguments: {tc.arguments}")
        
        # Execute using registry
        result = registry.execute(tc.name, **tc.arguments)
        print(f"  Result: {result.result}")

---
## 6. Exercises <a id='exercises'></a>

### Exercise 1: Multi-Tool Request

Ask a question that requires multiple tools and handle all the calls.

In [ ]:
# TODO: Send a request like "Compare the weather in Paris and London"
# Handle all tool calls and get the final response

# Your code here:


### Exercise 2: Add a New Tool

Add a `get_time` tool that returns the current time in a timezone.

In [ ]:
# TODO: Create a get_time tool
# Parameters: timezone (string)
# Returns current time in that timezone

# Your code here:


### Exercise 3: Error Recovery

Implement a flow that gracefully handles tool errors and asks the LLM to try a different approach.

In [ ]:
# TODO: Implement error recovery
# If a tool fails, send the error back to the LLM
# Let it decide what to do next

# Your code here:


---
## 7. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You understand both OpenAI and Anthropic tool formats
- [ ] You can handle multi-turn conversations with tools
- [ ] You implemented proper error handling
- [ ] You can use our LLMClient with tools
- [ ] You completed at least 2 exercises

### Next Steps

In the next notebook, we'll build **ReAct Agents** - putting together everything we've learned to create agents that can reason and act!

---
## Summary

**Key Concepts:**

1. **OpenAI** uses `tools` with `type: function` wrapper
2. **Anthropic** uses direct tool objects with `input_schema`
3. **Multi-turn** requires adding tool results back to messages
4. **Error handling** is essential for production systems
5. **Our LLMClient** abstracts provider differences

**Production Checklist:**
- Validate arguments before execution
- Handle unknown functions gracefully
- Set iteration limits
- Log all tool calls for debugging
- Return useful error messages to the LLM